In [9]:
from warnings import filterwarnings
filterwarnings('ignore')
import pandas as pd
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("PySparkTest").getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

dataset_path = '/media/arvis/AEAA2666AA262B73/pyspark_tutorial/datasets/'

## DataFrame Creation

In [10]:
data = [("Alice", 25), ("Bob", 30), ("Cathy", 29)]
columns = ["Name", "Age"]
df = spark.createDataFrame(data, columns)
df.show()

+-----+---+
| Name|Age|
+-----+---+
|Alice| 25|
|  Bob| 30|
|Cathy| 29|
+-----+---+



In [3]:
df.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Age: long (nullable = true)



## Importing csv file

In [4]:
dcsv = spark.read.csv(dataset_path+'spam_ham_dataset.csv',header=True,inferSchema=True)
dcsv.show()


+--------------------+--------------------+--------------------+--------------------+
|                 _c0|               label|                text|           label_num|
+--------------------+--------------------+--------------------+--------------------+
|                 605|                 ham|Subject: enron me...|                NULL|
|this is a follow ...| 4 / 3 / 00 { pre...|                NULL|                NULL|
|flow data provide...|                NULL|                NULL|                NULL|
|please override p...|                NULL|                NULL|                NULL|
|activity you can ...|                NULL|                NULL|                NULL|
|this change is ne...|                   0|                NULL|                NULL|
|                2349|                 ham|Subject: hpl nom ...|                NULL|
|( see attached fi...|                NULL|                NULL|                NULL|
|  - hplnol 09 . xls"|                   0|           

<b>Using Pandas</b>

In [5]:
pd.read_csv(dataset_path+"spam_ham_dataset.csv",index_col=None)

,Unnamed: 0,label,text,label_num
0,605,ham,Subject: enron methanol ; meter # : 988291\r\n...,0
1,2349,ham,"Subject: hpl nom for january 9 , 2001\r\n( see...",0
2,3624,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0
3,4685,spam,"Subject: photoshop , windows , office . cheap ...",1
4,2030,ham,Subject: re : indian springs\r\nthis deal is t...,0
...,...,...,...,...
5166,1518,ham,Subject: put the 10 on the ft\r\nthe transport...,0
5167,404,ham,Subject: 3 / 4 / 2000 and following noms\r\nhp...,0
5168,2933,ham,Subject: calpine daily gas nomination\r\n>\r\n...,0
5169,1409,ham,Subject: industrial worksheets for august 2000...,0


#### 

<b> Observation: </b> when used spark.read.csv() it yielded wrong formated data as compared to pandas

#### 

<b>:: Solution :: </b>

In [6]:
from pyspark.sql import SparkSession

# Start Spark session
spark = SparkSession.builder.appName("SpamHamSQL").getOrCreate()

# Load CSV into DataFrame
# df = spark.read.option("header", True).csv(dataset_path+"spam_ham_dataset.csv")
df = spark.read.option("header", True) \
               .option("multiLine", True) \
               .option("escape", "\"") \
               .option("quote", "\"") \
               .option("inferSchema", True) \
               .csv(dataset_path+"spam_ham_dataset.csv")
# Register as temporary SQL table (view)
df.createOrReplaceTempView("spam_ham")

# Run SQL query: Select first 5 rows
result = spark.sql("SELECT text,label_num FROM spam_ham")

# Show result
result.show()


+--------------------+---------+
|                text|label_num|
+--------------------+---------+
|Subject: enron me...|        0|
|Subject: hpl nom ...|        0|
|Subject: neon ret...|        0|
|Subject: photosho...|        1|
|Subject: re : ind...|        0|
|Subject: ehronlin...|        0|
|Subject: spring s...|        0|
|Subject: looking ...|        1|
|Subject: noms / a...|        0|
|Subject: nominati...|        0|
|Subject: vocable ...|        1|
|Subject: report 0...|        1|
|Subject: enron / ...|        0|
|Subject: vic . od...|        1|
|Subject: tenaska ...|        0|
|Subject: underpri...|        1|
|Subject: re : fir...|        0|
|Subject: swift - ...|        0|
|Subject: meter va...|        0|
|Subject: addition...|        0|
+--------------------+---------+
only showing top 20 rows


In [7]:
type(result)

pyspark.sql.classic.dataframe.DataFrame

<b>Unique Labels</b>

In [8]:
result.select('label_num').distinct().show()


+---------+
|label_num|
+---------+
|        1|
|        0|
+---------+

